Notebook to pre-process the metadata downloaded from the database of interventions in congreso.es after being concatenated by legislature. Adds a new feature called "political group".

In [1]:
import pandas as pd
import os
import glob
import re

In [2]:
workdir = 'data/interventions/merged-by-legislature'
title = 'all-interventions-clean.csv'
os.chdir(f'./{workdir}')

In [3]:
# Get all the file names.
filenames = [i for i in glob.glob('*.csv')]

files = []

for i in range(0, len(filenames)):
    files.append(pd.read_csv(filenames[i]))

# Concatenate all files in one.
data = pd.concat(files)

# Keep only useful fields.
data = data[['legislatura', 'fecha', 'objeto_iniciativa',
    'numero_expediente', 'autores', 'nombre_sesion',
    'orador', 'enlace_pdf']]

# Eliminate around 15 rows in L03 that are missplaced.
l = ['NUÑEZ ENCABO, MANUEL (GS)', 'MOYA PUEYO, VICENTE',
    'PEREZ RUBALCABA, ALFREDO', 'TOCINO BISCAROLASAGA, ISABEL (GCP)',
    'OLLERO TASSARA, ANDRES (APDP)',
    'MONTESINOS GARCIA, JUAN ANTONIO (GCP)',
    'CUENCA I VALERO, MARIA EUGENIA (GMC)',
    'GARCIA FONSECA, MANUEL (AIU-EC)', 'VILLAMOR LEON, JOSE']

data = data.drop(data.loc[data['fecha'].isin(l)].index)

# Eliminate 4 rows of data with errors.
l2 = ['COMPARECENCIA DE AUTORIDADES Y FUNCIONARIOS EN COMISION.', 
      'COMPARECENCIA DEL GOBIERNO EN COMISION (ART. 44).']

data = data.drop(data.loc[data['legislatura'].isin(l2)].index)

# Eliminating 2 rows in L06 that are missplaced.
data = data.loc[(data['fecha'] != 'Pregunta-Contestación')]

# Fecha to datetime format.
data['fecha'] = pd.to_datetime(data['fecha'], format='%d/%m/%Y')

# Removing the  page reference since it is not needed and does not allow to drop duplicates.
data = data.astype({'enlace_pdf':'string'})

# ERROR here. The digit itself is not being replaced.
data['enlace_pdf'] = data['enlace_pdf'].str.replace(r'\#page=[\d]{1,3}', '')

# Remove duplicates.
data = data.sort_values(by=['fecha', 'enlace_pdf']).drop_duplicates().reset_index(drop=True)

# Eliminate NaNs.
data = data.dropna()

# Eliminate rows if they correspond to constitution of commissions because these are irrelevant.
data = data[data['objeto_iniciativa'].str.contains('Constitución de la Comisión') == False]

# Reset index.
data = data.reset_index(drop=True)

# Substitute roman numbers for integers values.
data['legislatura'] = data['legislatura'].replace({'I': 1, 'II': 2, 'III': 3, 'IV': 4, 'V': 5, 'VI': 6, 'VII': 7, 'VIII': 8, 'IX': 9, 'X': 10, 'XI': 11, 'XII': 12, 'XIII': 13, 'XIV': 14})

# Adding the political group as a new feature.
political_group = data['orador'].str.split(r' \(', 1)

# Filling the cases where there is no political group.
for i in range(len(political_group)):
    try: political_group[i][1]
    except: political_group[i].append('')

# Clean the list to only include the political group, e.g. GP, GCUP-EC-GC, GS.
political_group = [political_group[i][1][:-1]
                   for i in range(len(political_group))]

# Add the column in the DF.
data['political_group'] = political_group

# Save new file.
data.to_csv(f'../{title}', index=False)

/var/folders/ct/921g5yd903j7f8wh5k85dltr0000gn/T/ipykernel_70550/3869542844.py:7: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  files.append(pd.read_csv(filenames[i]))
/var/folders/ct/921g5yd903j7f8wh5k85dltr0000gn/T/ipykernel_70550/3869542844.py:7: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  files.append(pd.read_csv(filenames[i]))
/var/folders/ct/921g5yd903j7f8wh5k85dltr0000gn/T/ipykernel_70550/3869542844.py:43: FutureWarning: The default value of regex will change from True to False in a future version.
  data['enlace_pdf'] = data['enlace_pdf'].str.replace(r'\#page=[\d]{1,3}', '')


In [5]:
# Since we can only obtain the texts for legislatures 6 until 14, I splitted this data into another .csv file.
vi_to_xiv = data.loc[data['legislatura'] > 5].reset_index(drop=True)
vi_to_xiv.to_csv(f'../vi-xiv-clean.csv', index=False)

In [4]:
# Obtaining a file per legislature.
vi   = data.loc[data['legislatura'] ==  6].reset_index(drop=True)
vii  = data.loc[data['legislatura'] ==  7].reset_index(drop=True)
viii = data.loc[data['legislatura'] ==  8].reset_index(drop=True)
ix   = data.loc[data['legislatura'] ==  9].reset_index(drop=True)
x    = data.loc[data['legislatura'] == 10].reset_index(drop=True)
xi   = data.loc[data['legislatura'] == 11].reset_index(drop=True)
xii  = data.loc[data['legislatura'] == 12].reset_index(drop=True)
xiii = data.loc[data['legislatura'] == 13].reset_index(drop=True)
xiv  = data.loc[data['legislatura'] == 14].reset_index(drop=True)

vi.to_csv(f'../L06-clean.csv', index=False)
vii.to_csv(f'../L07-clean.csv', index=False)
viii.to_csv(f'../L08-clean.csv', index=False)
ix.to_csv(f'../L09-clean.csv', index=False)
x.to_csv(f'../L10-clean.csv', index=False)
xi.to_csv(f'../L11-clean.csv', index=False)
xii.to_csv(f'../L12-clean.csv', index=False)
xiii.to_csv(f'../L13-clean.csv', index=False)
xiv.to_csv(f'../L14-clean.csv', index=False)